In [47]:
%pip install braindecode
from braindecode.datasets.bbci import  BBCIDataset
cnt = BBCIDataset(filename='/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/5.mat', load_sensor_names=None).load()


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


KeyError: "Unable to open object (object 'clab' doesn't exist)"

In [48]:
%pip install braindecode
from braindecode.datasets.bbci import BBCIDataset

# Carica il dataset
cnt = BBCIDataset(filename='/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/5.mat', load_sensor_names=None).load()

# Estrai la chiave 'clab' da 'mnt' o 'nfo' (se presenti)
if 'mnt' in cnt and 'clab' in cnt['mnt']:
    clab_values = cnt['mnt']['clab']
    print("Chiavi trovate in 'mnt':", clab_values)

if 'nfo' in cnt and 'clab' in cnt['nfo']:
    clab_values = cnt['nfo']['clab']
    print("Chiavi trovate in 'nfo':", clab_values)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


KeyError: "Unable to open object (object 'clab' doesn't exist)"

In [4]:
import h5py

with h5py.File("/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/test/1.mat", 'r') as f:
    print("Chiavi principali nel file:", list(f.keys()))

    # Se esiste una struttura "dat", controlliamo cosa contiene
    if 'dat' in f:
        print("Chiavi dentro 'dat':", list(f['dat'].keys()))
    
    # Se esiste una chiave che potrebbe contenere nomi di canali
    for key in f.keys():
        if isinstance(f[key], h5py.Group):  # Se è un gruppo di dati
            print(f"Chiavi dentro '{key}':", list(f[key].keys()))


Chiavi principali nel file: ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', '

In [8]:
import numpy as np
import os
import logging
from collections import OrderedDict
import h5py  # Usa h5py per caricare i file .mat in formato HDF5
import mne

def load_HGD(data_path, subject, training, low_cut_hz=0, debug=False):
    """Loading training/testing data for the High Gamma Dataset (HGD)
    for a specific subject.
    
    Parameters
    ----------
    data_path: string
        dataset path
    subject: int
        number of subject in [1, .. ,14]
    training: bool
        if True, load training data
        if False, load testing data
    low_cut_hz: float
        low frequency cutoff for high-pass filtering
    debug: bool
        if True, print debug information
    """
    
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')
    print("DEBUG: Inizio load_HGD_data")
    
    # Aggiungi la barra finale per il percorso
    if training:
        filename = os.path.join(data_path, 'train', f'{subject}.mat')
    else:
        filename = os.path.join(data_path, 'test', f'{subject}.mat')
    
    print(f"DEBUG: File da caricare: {filename}")
    
    load_sensor_names = None
    if debug:
        load_sensor_names = ['ch1']
    
    print(f"DEBUG: Sensor names da caricare: {load_sensor_names}")
    
    # Verifica se il file esiste
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Il file {filename} non esiste.")
    print("DEBUG: Il file esiste.")
    
    # Carica il file .mat usando h5py (per file HDF5)
    with h5py.File(filename, 'r') as mat_data:
        # Controlla le chiavi del file .mat
        print(f"DEBUG: Le chiavi nel file mat sono: {list(mat_data.keys())}")
        
        # Estrai i dati EEG e le etichette
        X = mat_data.get('data')  # I dati EEG dovrebbero essere in 'data'
        y = mat_data.get('labels')  # Le etichette dovrebbero essere in 'labels'
        
        if X is None or y is None:
            raise ValueError("I dati 'data' o 'labels' non sono presenti nel file .mat")
        
        X = np.array(X)  # Assicurati che X sia un array numpy
        y = np.array(y)  # Assicurati che y sia un array numpy
    
    # Verifica la forma dei dati
    print(f"DEBUG: X shape: {X.shape}, y shape: {y.shape}")
    
    # Seleziona solo i sensori necessari (C_sensors)
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5', 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 
                 'CCP6h', 'CPP5h', 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h', 'CCP1h', 'CCP2h', 'CPP1h', 'CPP2h']
    
    if debug:
        C_sensors = load_sensor_names  # Usa solo i sensori di debug
    
    # Crea un oggetto MNE Raw da X
    info = mne.create_info(ch_names=C_sensors, sfreq=250, ch_types='eeg')  # Frequenza di campionamento 250 Hz (modifica se necessario)
    raw = mne.io.RawArray(X.T, info)  # X deve essere trasposto per essere compatibile con MNE
    
    # Preprocessing dei dati come descritto nel paper
    log.info("Resampling...")
    raw.resample(250.0)  # Esegui il resampling a 250 Hz (modifica se necessario)
    
    log.info("Highpassing...")
    raw.filter(l_freq=low_cut_hz, h_freq=None)  # High-pass filtering
    
    log.info("Standardizing...")
    from braindecode.datautil.signalproc import exponential_running_standardize
    raw_data = exponential_running_standardize(raw.get_data().T, factor_new=1e-3, init_block_size=1000, eps=1e-4).T
    
    # Crea un nuovo RawArray con i dati preprocessati
    raw = mne.io.RawArray(raw_data, info)
    
    # Intervallo per il trial, inizia a -500, poiché aiuta nella decodifica migliorata
    ival = [-500, 4000]
    
    # Creazione dei dati di segnali e target
    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2]),
                              ('Rest', [3]), ('Feet', [4])])
    
    dataset = create_signal_target_from_raw_mne(raw, marker_def, ival)
    
    # Pulizia dei trial con i valori assoluti dei segnali superiori a 800
    clean_trial_mask = np.max(np.abs(dataset.X), axis=(1, 2)) < 800
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    
    return dataset.X, dataset.y

# Carica i dati di un soggetto
subject = 1
training = True 
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/"
X, y = load_HGD(data_path, subject, training, low_cut_hz=4, debug=True)
print(X.shape, y.shape)


DEBUG: Inizio load_HGD_data
DEBUG: File da caricare: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
DEBUG: Sensor names da caricare: ['ch1']
DEBUG: Il file esiste.
DEBUG: Le chiavi nel file mat sono: ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', '

ValueError: I dati 'data' o 'labels' non sono presenti nel file .mat

In [11]:
import h5py

file_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

# Apriamo il file in modalità lettura
with h5py.File(file_path, 'r') as h5file:
    data = {}
    
    # Scansioniamo tutte le chiavi principali
    for key in h5file.keys():
        obj = h5file[key]
        if isinstance(obj, h5py.Dataset):  # Se è un dataset, leggilo
            data[key] = obj[()]
        elif isinstance(obj, h5py.Group):  # Se è un gruppo, stampiamo le sottosezioni
            print(f"Il gruppo '{key}' contiene le seguenti chiavi: {list(obj.keys())}")

    # Cerchiamo 'clab'
    clab_data = None
    if 'nfo' in h5file and 'clab' in h5file['nfo']:
        clab_data = h5file['nfo']['clab'][()]
    elif 'mnt' in h5file and 'clab' in h5file['mnt']:
        clab_data = h5file['mnt']['clab'][()]
    
    if clab_data is not None:
        print("Clab trovato con successo:", clab_data)
    else:
        print("Errore: 'clab' non trovato in 'nfo' o 'mnt'.")


Il gruppo 'dat' contiene le seguenti chiavi: ['yUnit']
Il gruppo 'mnt' contiene le seguenti chiavi: ['box', 'box_sz', 'clab', 'pos_3d', 'scale_box', 'scale_box_sz', 'x', 'y']
Il gruppo 'mrk' contiene le seguenti chiavi: ['className', 'event', 'time', 'y']
Il gruppo 'nfo' contiene le seguenti chiavi: ['T', 'clab', 'className', 'fs']
Clab trovato con successo: [[<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object ref

In [17]:
import h5py

file_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(file_path, 'r') as h5file:
    clab_data = None

    def extract_clab(group_name):
        """ Estrae il contenuto di 'clab' da un gruppo se presente. """
        if group_name in h5file and 'clab' in h5file[group_name]:
            clab_entry = h5file[group_name]['clab']
            print(f"Tipo di {group_name}['clab']:", clab_entry.dtype)

            # Se il dataset contiene array di stringhe direttamente
            if clab_entry.dtype.kind in {'S', 'U'}:  
                return [s.decode('utf-8') if isinstance(s, bytes) else s for s in clab_entry[:]]

            # Se il dataset contiene un singolo valore stringa
            elif isinstance(clab_entry[()], bytes):
                return [clab_entry[()].decode('utf-8')]

            return None

    # Prova prima 'nfo', poi 'mnt'
    clab_data = extract_clab('nfo') or extract_clab('mnt')

    if clab_data:
        print("Clab risolto con successo:", clab_data)
    else:
        print("Errore: 'clab' non trovato o non interpretabile.")


Tipo di nfo['clab']: object
Tipo di mnt['clab']: object
Errore: 'clab' non trovato o non interpretabile.


In [25]:
import h5py

file_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(file_path, 'r') as h5file:
    def resolve_references(group_name):
        """Risolvi i riferimenti all'interno di 'clab'."""
        if group_name in h5file and 'clab' in h5file[group_name]:
            clab_entry = h5file[group_name]['clab']
            print(f"Contenuto di {group_name}['clab']:", clab_entry)

            # Verifica se i riferimenti sono oggetti HDF5
            if isinstance(clab_entry, h5py.Dataset):
                resolved_clab = []
                for ref in clab_entry:
                    if isinstance(ref, h5py.Reference):
                        # Risolvi il riferimento HDF5 e accedi ai dati
                        resolved_clab.append(h5file[ref][()])
                    else:
                        resolved_clab.append(ref)
                return resolved_clab
        return None

    # Risolvi i riferimenti in 'nfo' e 'mnt'
    nfo_clab_resolved = resolve_references('nfo')
    mnt_clab_resolved = resolve_references('mnt')

    if nfo_clab_resolved is not None:
        print("Contenuto di 'nfo['clab']' risolto:", nfo_clab_resolved)
    if mnt_clab_resolved is not None:
        print("Contenuto di 'mnt['clab']' risolto:", mnt_clab_resolved)
        
    



Contenuto di nfo['clab']: <HDF5 dataset "clab": shape (133, 1), type "|O">
Contenuto di mnt['clab']: <HDF5 dataset "clab": shape (133, 1), type "|O">
Contenuto di 'nfo['clab']' risolto: [array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object

In [35]:
file1 = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

#Apriamo il file in modalità lettura
with h5py.File(file1, 'r') as h5file:
    data = {}
    
    #Voglio creare un dataset con i dati di 'data'
    for key in h5file.keys():
        obj = h5file[key]
        if isinstance(obj, h5py.Dataset):
            data[key] = obj[()]
        elif isinstance(obj, h5py.Group):
            print(f"Il gruppo '{key}' contiene le seguenti chiavi: {list(obj.keys())}")

#Stampo i dati
print(data)

#Creo un dataframe con i dati
import pandas as pd
df = pd.DataFrame(data)
print(df)

Il gruppo 'dat' contiene le seguenti chiavi: ['yUnit']
Il gruppo 'mnt' contiene le seguenti chiavi: ['box', 'box_sz', 'clab', 'pos_3d', 'scale_box', 'scale_box_sz', 'x', 'y']
Il gruppo 'mrk' contiene le seguenti chiavi: ['className', 'event', 'time', 'y']
Il gruppo 'nfo' contiene le seguenti chiavi: ['T', 'clab', 'className', 'fs']
{'ch1': array([[-43.988, -43.78 , -42.913, ...,   1.764,   3.395,   3.633]]), 'ch10': array([[-35.77 , -34.399, -35.828, ..., -20.85 , -22.11 , -23.592]]), 'ch100': array([[189.288, 149.009, 163.819, ..., 179.694, 182.462, 183.44 ]]), 'ch101': array([[153.225, 148.786, 163.482, ..., 152.385, 156.136, 157.215]]), 'ch102': array([[-147.787, -147.784, -147.033, ..., -117.112, -116.691, -116.893]]), 'ch103': array([[-155.45 , -154.199, -151.753, ..., -131.341, -130.961, -130.333]]), 'ch104': array([[-128.697, -126.186, -134.103, ...,  -99.69 , -100.094, -102.728]]), 'ch105': array([[-124.949, -118.963, -110.028, ...,  -90.904,  -89.395,  -87.949]]), 'ch106': arr

ValueError: Per-column arrays must each be 1-dimensional

In [37]:
import h5py
import pandas as pd
import numpy as np

file1 = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

# Apriamo il file in modalità lettura
with h5py.File(file1, 'r') as h5file:
    data = {}
    
    # Iteriamo attraverso le chiavi per estrarre i dati
    for key in h5file.keys():
        obj = h5file[key]
        
        # Se l'oggetto è un dataset, otteniamo i suoi dati
        if isinstance(obj, h5py.Dataset):
            data[key] = obj[()]
        elif isinstance(obj, h5py.Group):
            print(f"Il gruppo '{key}' contiene le seguenti chiavi: {list(obj.keys())}")
    
# Stampa della struttura dei dati
print("Struttura dei dati estratti:")
for key, value in data.items():
    print(f"{key}: {type(value)} con forma {value.shape if hasattr(value, 'shape') else 'N/A'}")

# Prepara i dati per il DataFrame
flat_data = {}
max_length = None

# Trova la lunghezza massima tra i dataset
for key, value in data.items():
    if isinstance(value, np.ndarray) and value.ndim > 1:
        # Appiattiamo l'array se è multidimensionale
        flat_data[key] = value.flatten()
    else:
        flat_data[key] = value

    # Verifica e salva la lunghezza massima per il DataFrame
    if max_length is None:
        max_length = len(flat_data[key])
    elif len(flat_data[key]) != max_length:
        print(f"Attenzione: la lunghezza di '{key}' è {len(flat_data[key])}, che è diversa da {max_length}")

# Creiamo il DataFrame solo se tutte le colonne hanno la stessa lunghezza
if all(len(v) == max_length for v in flat_data.values()):
    df = pd.DataFrame(flat_data)
    print("DataFrame risultante:")
    print(df)
else:
    print("Errore: le lunghezze delle colonne sono diverse. Non posso creare il DataFrame.")


Il gruppo 'dat' contiene le seguenti chiavi: ['yUnit']
Il gruppo 'mnt' contiene le seguenti chiavi: ['box', 'box_sz', 'clab', 'pos_3d', 'scale_box', 'scale_box_sz', 'x', 'y']
Il gruppo 'mrk' contiene le seguenti chiavi: ['className', 'event', 'time', 'y']
Il gruppo 'nfo' contiene le seguenti chiavi: ['T', 'clab', 'className', 'fs']
Struttura dei dati estratti:
ch1: <class 'numpy.ndarray'> con forma (1, 1225545)
ch10: <class 'numpy.ndarray'> con forma (1, 1225545)
ch100: <class 'numpy.ndarray'> con forma (1, 1225545)
ch101: <class 'numpy.ndarray'> con forma (1, 1225545)
ch102: <class 'numpy.ndarray'> con forma (1, 1225545)
ch103: <class 'numpy.ndarray'> con forma (1, 1225545)
ch104: <class 'numpy.ndarray'> con forma (1, 1225545)
ch105: <class 'numpy.ndarray'> con forma (1, 1225545)
ch106: <class 'numpy.ndarray'> con forma (1, 1225545)
ch107: <class 'numpy.ndarray'> con forma (1, 1225545)
ch108: <class 'numpy.ndarray'> con forma (1, 1225545)
ch109: <class 'numpy.ndarray'> con forma (1, 1

In [38]:
import h5py
import numpy as np

file1 = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

# Apriamo il file in modalità lettura
with h5py.File(file1, 'r') as h5file:
    data = {}
    
    # Iteriamo attraverso le chiavi per estrarre i dati
    for key in h5file.keys():
        obj = h5file[key]
        
        # Se l'oggetto è un dataset, otteniamo i suoi dati
        if isinstance(obj, h5py.Dataset):
            data[key] = obj[()]
        elif isinstance(obj, h5py.Group):
            print(f"Il gruppo '{key}' contiene le seguenti chiavi: {list(obj.keys())}")

# Stampa la struttura dei dati
print("\nStruttura dei dati estratti:")
for key, value in data.items():
    print(f"{key}: {type(value)} con forma {value.shape if hasattr(value, 'shape') else 'N/A'}")

    # Esploriamo una parte dei dati
    if isinstance(value, np.ndarray):
        print(f"  Esempio di dati per '{key}': {value[:5]} ...")  # Mostriamo i primi 5 elementi

# Se ci sono array di lunghezze diverse, verifichiamo la lunghezza di ciascuno
for key, value in data.items():
    if isinstance(value, np.ndarray):
        print(f"\nLunghezza di '{key}': {len(value)}")
        print(f"Dettagli di '{key}':\n{value}")


Il gruppo 'dat' contiene le seguenti chiavi: ['yUnit']
Il gruppo 'mnt' contiene le seguenti chiavi: ['box', 'box_sz', 'clab', 'pos_3d', 'scale_box', 'scale_box_sz', 'x', 'y']
Il gruppo 'mrk' contiene le seguenti chiavi: ['className', 'event', 'time', 'y']
Il gruppo 'nfo' contiene le seguenti chiavi: ['T', 'clab', 'className', 'fs']

Struttura dei dati estratti:
ch1: <class 'numpy.ndarray'> con forma (1, 1225545)
  Esempio di dati per 'ch1': [[-43.988 -43.78  -42.913 ...   1.764   3.395   3.633]] ...
ch10: <class 'numpy.ndarray'> con forma (1, 1225545)
  Esempio di dati per 'ch10': [[-35.77  -34.399 -35.828 ... -20.85  -22.11  -23.592]] ...
ch100: <class 'numpy.ndarray'> con forma (1, 1225545)
  Esempio di dati per 'ch100': [[189.288 149.009 163.819 ... 179.694 182.462 183.44 ]] ...
ch101: <class 'numpy.ndarray'> con forma (1, 1225545)
  Esempio di dati per 'ch101': [[153.225 148.786 163.482 ... 152.385 156.136 157.215]] ...
ch102: <class 'numpy.ndarray'> con forma (1, 1225545)
  Esempi

In [49]:
import h5py

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/5.mat"

# Apri il file .mat e mostra le chiavi principali
with h5py.File(filename, 'r') as f:
    print("Chiavi principali nel file:", list(f.keys()))


Chiavi principali nel file: ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', '

In [50]:
with h5py.File(filename, 'r') as f:
    if 'mnt' in f:
        print("Chiavi in 'mnt':", list(f['mnt'].keys()))
    if 'nfo' in f:
        print("Chiavi in 'nfo':", list(f['nfo'].keys()))


Chiavi in 'mnt': ['box', 'box_sz', 'clab', 'pos_3d', 'scale_box', 'scale_box_sz', 'x', 'y']
Chiavi in 'nfo': ['T', 'clab', 'className', 'fs']


In [51]:
def read_clab(filename):
    with h5py.File(filename, 'r') as f:
        clab_data = None

        if 'mnt' in f and 'clab' in f['mnt']:
            clab_data = f['mnt']['clab'][:]
        elif 'nfo' in f and 'clab' in f['nfo']:
            clab_data = f['nfo']['clab'][:]
        
        if clab_data is not None:
            # Convertiamo i dati in stringhe leggibili
            clab_data = ["".join(chr(c) for c in f[ref]) for ref in clab_data]
            return clab_data
        else:
            print("Chiave 'clab' non trovata.")
            return None

clab_values = read_clab(filename)
print("Valori di 'clab':", clab_values)


TypeError: Accessing a group is done with bytes or str, not <class 'numpy.ndarray'>

In [52]:
import h5py
import numpy as np

def read_clab(filename):
    with h5py.File(filename, 'r') as f:
        clab_data = None

        if 'mnt' in f and 'clab' in f['mnt']:
            clab_data = f['mnt']['clab'][:]
        elif 'nfo' in f and 'clab' in f['nfo']:
            clab_data = f['nfo']['clab'][:]
        
        if clab_data is not None:
            # Se `clab_data` è un array di byte, convertiamolo in stringhe
            if isinstance(clab_data, np.ndarray):
                clab_data = [s.decode('utf-8') if isinstance(s, bytes) else str(s) for s in clab_data]
            
            return clab_data
        else:
            print("Chiave 'clab' non trovata.")
            return None

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/5.mat"
clab_values = read_clab(filename)
print("Valori di 'clab':", clab_values)


Valori di 'clab': ['[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference>]', '[<HDF5 object reference

In [57]:
import h5py
import numpy as np

def read_clab(filename):
    with h5py.File(filename, 'r') as f:
        clab_data = None

        # Controlliamo dove si trova 'clab'
        if 'mnt' in f and 'clab' in f['mnt']:
            clab_data = f['mnt']['clab'][:]
        elif 'nfo' in f and 'clab' in f['nfo']:
            clab_data = f['nfo']['clab'][:]

        if clab_data is not None:
            clab_strings = []

            for ref in clab_data:
                if isinstance(ref, h5py.Reference):  # Verifichiamo se è un riferimento HDF5
                    obj = f[ref]  # Accediamo all'oggetto referenziato

                    if isinstance(obj, h5py.Dataset):
                        data = obj[:]
                        
                        if isinstance(data, np.ndarray) and data.dtype == np.uint8:  # Array di byte
                            string_value = data.tobytes().decode('utf-8')  # Convertiamo in stringa
                            clab_strings.append(string_value)
                        else:
                            clab_strings.append(str(data))
                    else:
                        clab_strings.append(f"Tipo non gestito: {type(obj)}")
                else:
                    clab_strings.append(f"Non è un riferimento valido: {ref}")

            return clab_strings
        else:
            print("Chiave 'clab' non trovata.")
            return None

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/test/8.mat"
clab_values = read_clab(filename)
print("Valori di 'clab':", clab_values)


Valori di 'clab': ['Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference>]', 'Non è un riferimento valido: [<HDF5 object reference

In [64]:
import h5py

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"  # Cambia il numero in base al soggetto
#Ottengo le chiavi disponibili nel file .mat
with h5py.File(filename, 'r') as f:
    print("Chiavi disponibili nel file .mat:", list(f.keys()))

Chiavi disponibili nel file .mat: ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch

In [80]:
import h5py
import numpy as np

def load_HGD_Manuale(data_path, subject, training, low_cut_hz=0, debug=True):
    """Caricamento manuale del dataset HGD con h5py"""

    # Determina il file corretto
    if training:
        filename = f"{data_path}/train/{subject}.mat"
    else:
        filename = f"{data_path}/test/{subject}.mat"
    
    print(f"Caricamento file: {filename}")

    # Apri il file HDF5
    with h5py.File(filename, 'r') as f:
        # Controlla dove si trovano le informazioni sui canali
        if 'nfo' in f and 'clab' in f['nfo']:
            channel_labels = list(f['nfo']['clab'])  # Non è necessario fare la decodifica
        elif 'mnt' in f and 'clab' in f['mnt']:
            channel_labels = list(f['mnt']['clab'])  # Non è necessario fare la decodifica
        else:
            raise KeyError("Chiave 'clab' non trovata in 'nfo' o 'mnt'!")

        print("Canali EEG trovati:", channel_labels)

        # Controlliamo dove sono i dati EEG
        if 'dat' in f:
            eeg_data = np.array(f['dat'])
            print(f"Dati EEG trovati con forma: {eeg_data.shape}")
        else:
            raise KeyError("Chiave 'dat' non trovata nel file .mat!")

        # Selezioniamo solo i canali EEG richiesti
        C_sensors = ['C3', 'C4', 'C2']
        eeg_indices = [i for i, ch in enumerate(channel_labels) if ch in C_sensors]

        eeg_selected = eeg_data[:, eeg_indices] if eeg_indices else eeg_data
        print(f"Dati EEG selezionati con forma: {eeg_selected.shape}")

    return eeg_selected, channel_labels

In [81]:
# Test della funzione load_HGD_dataManuale
subject = 1
training = True
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/"

X, channel_labels = load_HGD_Manuale(data_path, subject, training, low_cut_hz=4, debug=True)

print("Forma dei dati EEG:", X.shape)
print("Canali EEG:", channel_labels)

Caricamento file: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset//train/1.mat
Canali EEG trovati: [array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object), array([<HDF5 object reference>], dtype=object),

In [83]:
import h5py

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(filename, "r") as f:
    print(list(f.keys()))  # Elenco delle variabili salvate


['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', 'ch86', 'ch87', 'ch88', 'ch89

In [85]:
filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"
with h5py.File(filename, 'r') as h5file:
    all_sensor_names = list(h5file.keys())  # Visualizza tutte le chiavi del file
    print(all_sensor_names)


['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', 'ch86', 'ch87', 'ch88', 'ch89

In [86]:
import h5py

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"
with h5py.File(filename, 'r') as f:
    print(f.keys()) 

<KeysViewHDF5 ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', 'ch86', 'ch87',

In [88]:
import h5py
import numpy as np

file_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(file_path, 'r') as f:
    # Visualizza tutte le chiavi principali
    print(f.keys())  # Ad esempio, 'ch1', 'ch2', ecc.
    
    # Carica i dati EEG da tutte le chiavi che iniziano con 'ch'
    eeg_data = []
    for key in f.keys():
        if key.startswith('ch'):
            eeg_data.append(f[key][:])
    
    # Concatena i dati EEG lungo l'asse appropriato
    eeg_data = np.concatenate(eeg_data, axis=0)
    
    print("Dati EEG caricati correttamente.")
    print(eeg_data.shape)  # Mostra la forma dei dati EEG caricati

<KeysViewHDF5 ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', 'ch86', 'ch87',

In [92]:
import os
import h5py
import numpy as np
import logging
from collections import OrderedDict
import mne
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, highpass_cnt

def load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True):
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')
    
    print("DEBUG: Inizio load_HGD_data")
    
    if training:
        filename = os.path.join(data_path, 'train', f'{subject}.mat')
    else:
        filename = os.path.join(data_path, 'test', f'{subject}.mat')
    print(f"DEBUG: File da caricare: {filename}")

    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']
    print(f"DEBUG: Sensor names da caricare: {load_sensor_names}")

    # Verifica se il file esiste
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Il file {filename} non esiste.")
    print("DEBUG: Il file esiste.")
    
    # Carica il file .mat
    try:
        with h5py.File(filename, 'r') as f:
            print("DEBUG: Contenuto del file .mat:")
            for key in f.keys():
                print(key)
            
            # Carica i dati EEG da tutte le chiavi che iniziano con 'ch'
            eeg_data = []
            for key in f.keys():
                if key.startswith('ch'):
                    eeg_data.append(f[key][:])
            
            # Concatena i dati EEG lungo l'asse appropriato
            eeg_data = np.concatenate(eeg_data, axis=0)
            
            # Carica gli eventi se presenti
            if 'mrk' in f:
                events = f['mrk'][:]
            else:
                raise KeyError(f"L'oggetto 'mrk' non esiste nel file {filename}.")
    except Exception as e:
        raise OSError(f"Errore durante l'apertura del file {filename}: {e}")
    
    print("DEBUG: Caricamento dati con h5py completato.")
    print("Dati EEG caricati correttamente.")
    print(eeg_data.shape)  # Mostra la forma dei dati EEG caricati
    
    # Continua con il resto del codice per elaborare i dati caricati
    # Ad esempio, puoi convertire eeg_data in un oggetto RawArray di MNE
    info = mne.create_info(ch_names=[f'ch{i}' for i in range(1, eeg_data.shape[0] + 1)], sfreq=250, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data, info)
    
    # Aggiungi gli eventi al raw
    events = np.array(events)
    event_id = {'Right Hand': 1, 'Left Hand': 2, 'Rest': 3, 'Feet': 4}
    raw.add_events(events, stim_channel='STI 014')
    
    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later
    log.info("Cutting trials...")

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2]),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(raw, marker_def, clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    raw.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    log.info("Resampling...")
    raw.resample(250.0)
    log.info("Highpassing...")
    raw.filter(l_freq=low_cut_hz, h_freq=None)
    log.info("Standardizing...")
    raw.apply_function(lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                                 init_block_size=1000,
                                                                 eps=1e-4).T)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(raw, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset.X, dataset.y

# Esegui la funzione load_HGD_data
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset"
subject = 1
training = True
X, y = load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True)
print(X.shape)
print(y.shape)

DEBUG: Inizio load_HGD_data
DEBUG: File da caricare: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
DEBUG: Sensor names da caricare: ['C3', 'C4', 'C2']
DEBUG: Il file esiste.
DEBUG: Contenuto del file .mat:
ch1
ch10
ch100
ch101
ch102
ch103
ch104
ch105
ch106
ch107
ch108
ch109
ch11
ch110
ch111
ch112
ch113
ch114
ch115
ch116
ch117
ch118
ch119
ch12
ch120
ch121
ch122
ch123
ch124
ch125
ch126
ch127
ch128
ch129
ch13
ch130
ch131
ch132
ch133
ch14
ch15
ch16
ch17
ch18
ch19
ch2
ch20
ch21
ch22
ch23
ch24
ch25
ch26
ch27
ch28
ch29
ch3
ch30
ch31
ch32
ch33
ch34
ch35
ch36
ch37
ch38
ch39
ch4
ch40
ch41
ch42
ch43
ch44
ch45
ch46
ch47
ch48
ch49
ch5
ch50
ch51
ch52
ch53
ch54
ch55
ch56
ch57
ch58
ch59
ch6
ch60
ch61
ch62
ch63
ch64
ch65
ch66
ch67
ch68
ch69
ch7
ch70
ch71
ch72
ch73
ch74
ch75
ch76
ch77
ch78
ch79
ch8
ch80
ch81
ch82
ch83
ch84
ch85
ch86
ch87
ch88
ch89
ch9
ch90
ch91
ch92
ch93
ch94
ch95
ch96
ch97
ch98
ch99
dat
mnt
mrk
nfo
~obj_pointed_by_100280
~obj_pointed_b

OSError: Errore durante l'apertura del file /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat: Accessing a group is done with bytes or str, not <class 'slice'>

In [95]:
import os
import h5py
import numpy as np
import logging
from collections import OrderedDict
import mne
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, highpass_cnt

def load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True):
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')
    
    print("DEBUG: Inizio load_HGD_data")
    
    if training:
        filename = os.path.join(data_path, 'train', f'{subject}.mat')
    else:
        filename = os.path.join(data_path, 'test', f'{subject}.mat')
    print(f"DEBUG: File da caricare: {filename}")

    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']
    print(f"DEBUG: Sensor names da caricare: {load_sensor_names}")

    # Verifica se il file esiste
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Il file {filename} non esiste.")
    print("DEBUG: Il file esiste.")
    
    # Carica il file .mat
    try:
        with h5py.File(filename, 'r') as f:
            print("DEBUG: Contenuto del file .mat:")
            for key in f.keys():
                print(key)
            
            # Carica i dati EEG da tutte le chiavi che iniziano con 'ch'
            eeg_data = []
            for key in f.keys():
                if key.startswith('ch'):
                    eeg_data.append(f[key][:])
            
            # Concatena i dati EEG lungo l'asse appropriato
            eeg_data = np.concatenate(eeg_data, axis=0)
            
            # Carica gli eventi se presenti
            if 'mrk' in f:
                mrk_group = f['mrk']
                print("DEBUG: Contenuto del gruppo 'mrk':")
                for key in mrk_group.keys():
                    print(key)
                
                # Assumendo che gli eventi siano memorizzati sotto 'time' e 'event'
                events = np.array(mrk_group['time'])
                event_labels = np.array(mrk_group['event'])
                
                # Verifica le dimensioni degli array
                print(f"DEBUG: Dimensioni events: {events.shape}")
                print(f"DEBUG: Dimensioni event_labels: {event_labels.shape}")
                
                # Assicurati che gli array abbiano la stessa lunghezza lungo l'asse di concatenazione
                if events.shape[0] == event_labels.shape[0]:
                    events = np.column_stack((events, event_labels))
                else:
                    raise ValueError(f"Le dimensioni degli array events e event_labels non corrispondono: {events.shape[0]} vs {event_labels.shape[0]}")
            else:
                raise KeyError(f"L'oggetto 'mrk' non esiste nel file {filename}.")
    except Exception as e:
        raise OSError(f"Errore durante l'apertura del file {filename}: {e}")
    
    print("DEBUG: Caricamento dati con h5py completato.")
    print("Dati EEG caricati correttamente.")
    print(eeg_data.shape)  # Mostra la forma dei dati EEG caricati
    
    # Continua con il resto del codice per elaborare i dati caricati
    # Ad esempio, puoi convertire eeg_data in un oggetto RawArray di MNE
    info = mne.create_info(ch_names=[f'ch{i}' for i in range(1, eeg_data.shape[0] + 1)], sfreq=250, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data, info)
    
    # Aggiungi gli eventi al raw
    events = np.array(events)
    event_id = {'Right Hand': 1, 'Left Hand': 2, 'Rest': 3, 'Feet': 4}
    raw.add_events(events, stim_channel='STI 014')
    
    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later
    log.info("Cutting trials...")

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2]),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(raw, marker_def, clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    raw.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    log.info("Resampling...")
    raw.resample(250.0)
    log.info("Highpassing...")
    raw.filter(l_freq=low_cut_hz, h_freq=None)
    log.info("Standardizing...")
    raw.apply_function(lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                                 init_block_size=1000,
                                                                 eps=1e-4).T)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(raw, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset.X, dataset.y

# Esegui la funzione load_HGD_data
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset"
subject = 1
training = True
X, y = load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True)
print(X.shape)
print(y.shape)

DEBUG: Inizio load_HGD_data
DEBUG: File da caricare: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
DEBUG: Sensor names da caricare: ['C3', 'C4', 'C2']
DEBUG: Il file esiste.
DEBUG: Contenuto del file .mat:
ch1
ch10
ch100
ch101
ch102
ch103
ch104
ch105
ch106
ch107
ch108
ch109
ch11
ch110
ch111
ch112
ch113
ch114
ch115
ch116
ch117
ch118
ch119
ch12
ch120
ch121
ch122
ch123
ch124
ch125
ch126
ch127
ch128
ch129
ch13
ch130
ch131
ch132
ch133
ch14
ch15
ch16
ch17
ch18
ch19
ch2
ch20
ch21
ch22
ch23
ch24
ch25
ch26
ch27
ch28
ch29
ch3
ch30
ch31
ch32
ch33
ch34
ch35
ch36
ch37
ch38
ch39
ch4
ch40
ch41
ch42
ch43
ch44
ch45
ch46
ch47
ch48
ch49
ch5
ch50
ch51
ch52
ch53
ch54
ch55
ch56
ch57
ch58
ch59
ch6
ch60
ch61
ch62
ch63
ch64
ch65
ch66
ch67
ch68
ch69
ch7
ch70
ch71
ch72
ch73
ch74
ch75
ch76
ch77
ch78
ch79
ch8
ch80
ch81
ch82
ch83
ch84
ch85
ch86
ch87
ch88
ch89
ch9
ch90
ch91
ch92
ch93
ch94
ch95
ch96
ch97
ch98
ch99
dat
mnt
mrk
nfo
~obj_pointed_by_100280
~obj_pointed_b

OSError: Errore durante l'apertura del file /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat: Le dimensioni degli array events e event_labels non corrispondono: 320 vs 1

In [105]:
import os
import h5py
import logging
import numpy as np
import mne
from collections import OrderedDict
from braindecode.datasets.bbci import BBCIDataset
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, highpass_cnt

def load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True):
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')
    
    print("DEBUG: Inizio load_HGD_data")
    
    if training:
        filename = os.path.join(data_path, 'train', f'{subject}.mat')
    else:
        filename = os.path.join(data_path, 'test', f'{subject}.mat')
    print(f"DEBUG: File da caricare: {filename}")

    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']
    print(f"DEBUG: Sensor names da caricare: {load_sensor_names}")

    # Verifica se il file esiste
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Il file {filename} non esiste.")
    print("DEBUG: Il file esiste.")
    
    # Carica il file .mat
    try:
        with h5py.File(filename, 'r') as f:
            print("DEBUG: Contenuto del file .mat:")
            for key in f.keys():
                print(key)
            
            # Carica i dati EEG da tutte le chiavi che iniziano con 'ch'
            eeg_data = []
            for key in f.keys():
                if key.startswith('ch'):
                    eeg_data.append(f[key][:])
            
            # Concatena i dati EEG lungo l'asse appropriato
            eeg_data = np.concatenate(eeg_data, axis=0)
            
            # Carica gli eventi se presenti
            if 'mrk' in f:
                mrk_group = f['mrk']
                print("DEBUG: Contenuto del gruppo 'mrk':")
                for key in mrk_group.keys():
                    print(key)
                
                # Assumendo che gli eventi siano memorizzati sotto 'time' e 'event'
                events = np.array(mrk_group['time']).flatten()
                event_labels = np.array(mrk_group['event']).flatten()
                
                # Verifica le dimensioni degli array
                print(f"DEBUG: Dimensioni events: {events.shape}")
                print(f"DEBUG: Dimensioni event_labels: {event_labels.shape}")
                
                # Se event_labels ha una dimensione errata, accedi agli elementi interni
                if event_labels.shape[0] == 1 and isinstance(event_labels[0], np.ndarray):
                    event_labels = event_labels[0]
                    print(f"DEBUG: Nuove dimensioni event_labels: {event_labels.shape}")
                
                # Assicurati che gli array abbiano la stessa lunghezza lungo l'asse di concatenazione
                if events.shape[0] != event_labels.shape[0]:
                    # Adjust the length of event_labels to match events
                    event_labels = np.resize(event_labels, events.shape[0])
                    print(f"DEBUG: Nuove dimensioni event_labels dopo resize: {event_labels.shape}")
                
                # Converti gli eventi in interi
                events = events.astype(int)
                event_labels = event_labels.astype(int)
                
                # Create the events array with the correct shape
                events = np.column_stack((events, np.zeros(events.shape[0], dtype=int), event_labels))
            else:
                raise KeyError(f"L'oggetto 'mrk' non esiste nel file {filename}.")
    except Exception as e:
        raise OSError(f"Errore durante l'apertura del file {filename}: {e}")
    
    print("DEBUG: Caricamento dati con h5py completato.")
    print("Dati EEG caricati correttamente.")
    print(eeg_data.shape)  # Mostra la forma dei dati EEG caricati
    
    # Continua con il resto del codice per elaborare i dati caricati
    # Ad esempio, puoi convertire eeg_data in un oggetto RawArray di MNE
    info = mne.create_info(ch_names=[f'ch{i}' for i in range(1, eeg_data.shape[0] + 1)], sfreq=250, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data, info)
    
    # Aggiungi gli eventi al raw
    events = np.array(events)
    event_id = {'Right Hand': 1, 'Left Hand': 2, 'Rest': 3, 'Feet': 4}
    # Create a new channel for the events
    stim_data = np.zeros((1, raw.n_times))
    stim_info = mne.create_info(['STI 014'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, stim_info)
    
    # Add the stim channel to the raw object
    raw.add_channels([stim_raw], force_update_info=True)
    
    # Add the events to the stim channel
    raw.add_events(events, stim_channel='STI 014')
    
    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later
    log.info("Cutting trials...")

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2]),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(raw, marker_def, clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    raw.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    log.info("Resampling...")
    raw.resample(250.0)
    log.info("Highpassing...")
    raw.filter(l_freq=low_cut_hz, h_freq=None)
    log.info("Standardizing...")
    raw.apply_function(lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                                 init_block_size=1000,
                                                                 eps=1e-4).T)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(raw, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset.X, dataset.y

# Esegui la funzione load_HGD_data
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset"
subject = 1
training = True
X, y = load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True)
print(X.shape)
print(y.shape)

DEBUG: Inizio load_HGD_data
DEBUG: File da caricare: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
DEBUG: Sensor names da caricare: ['C3', 'C4', 'C2']
DEBUG: Il file esiste.
DEBUG: Contenuto del file .mat:
ch1
ch10
ch100
ch101
ch102
ch103
ch104
ch105
ch106
ch107
ch108
ch109
ch11
ch110
ch111
ch112
ch113
ch114
ch115
ch116
ch117
ch118
ch119
ch12
ch120
ch121
ch122
ch123
ch124
ch125
ch126
ch127
ch128
ch129
ch13
ch130
ch131
ch132
ch133
ch14
ch15
ch16
ch17
ch18
ch19
ch2
ch20
ch21
ch22
ch23
ch24
ch25
ch26
ch27
ch28
ch29
ch3
ch30
ch31
ch32
ch33
ch34
ch35
ch36
ch37
ch38
ch39
ch4
ch40
ch41
ch42
ch43
ch44
ch45
ch46
ch47
ch48
ch49
ch5
ch50
ch51
ch52
ch53
ch54
ch55
ch56
ch57
ch58
ch59
ch6
ch60
ch61
ch62
ch63
ch64
ch65
ch66
ch67
ch68
ch69
ch7
ch70
ch71
ch72
ch73
ch74
ch75
ch76
ch77
ch78
ch79
ch8
ch80
ch81
ch82
ch83
ch84
ch85
ch86
ch87
ch88
ch89
ch9
ch90
ch91
ch92
ch93
ch94
ch95
ch96
ch97
ch98
ch99
dat
mnt
mrk
nfo
~obj_pointed_by_100280
~obj_pointed_b

OSError: Errore durante l'apertura del file /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat: invalid literal for int() with base 10: 'desc'

In [104]:
import os
import h5py
import numpy as np
import logging
from collections import OrderedDict
import mne
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, highpass_cnt

def load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True):
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')
    
    print("DEBUG: Inizio load_HGD_data")
    
    if training:
        filename = os.path.join(data_path, 'train', f'{subject}.mat')
    else:
        filename = os.path.join(data_path, 'test', f'{subject}.mat')
    print(f"DEBUG: File da caricare: {filename}")

    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']
    print(f"DEBUG: Sensor names da caricare: {load_sensor_names}")

    # Verifica se il file esiste
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Il file {filename} non esiste.")
    print("DEBUG: Il file esiste.")
    
    # Carica il file .mat
    try:
        with h5py.File(filename, 'r') as f:
            print("DEBUG: Contenuto del file .mat:")
            for key in f.keys():
                print(key)
            
            # Carica i dati EEG da tutte le chiavi che iniziano con 'ch'
            eeg_data = []
            for key in f.keys():
                if key.startswith('ch'):
                    eeg_data.append(f[key][:])
            
            # Concatena i dati EEG lungo l'asse appropriato
            eeg_data = np.concatenate(eeg_data, axis=0)
            
            # Carica gli eventi se presenti
            if 'mrk' in f:
                mrk_group = f['mrk']
                print("DEBUG: Contenuto del gruppo 'mrk':")
                for key in mrk_group.keys():
                    print(key)
                
                # Assumendo che gli eventi siano memorizzati sotto 'time' e 'event'
                events = np.array(mrk_group['time']).flatten()
                event_labels = np.array(mrk_group['event']).flatten()
                
                # Verifica le dimensioni degli array
                print(f"DEBUG: Dimensioni events: {events.shape}")
                print(f"DEBUG: Dimensioni event_labels: {event_labels.shape}")
                
                # Se event_labels ha una dimensione errata, accedi agli elementi interni
                if event_labels.shape[0] == 1 and isinstance(event_labels[0], np.ndarray):
                    event_labels = event_labels[0]
                    print(f"DEBUG: Nuove dimensioni event_labels: {event_labels.shape}")
                
                # Assicurati che gli array abbiano la stessa lunghezza lungo l'asse di concatenazione
                if events.shape[0] != event_labels.shape[0]:
                    # Adjust the length of event_labels to match events
                    event_labels = np.resize(event_labels, events.shape[0])
                    print(f"DEBUG: Nuove dimensioni event_labels dopo resize: {event_labels.shape}")
                
                # Create the events array with the correct shape
                events = np.column_stack((events, np.zeros(events.shape[0], dtype=int), event_labels))
            else:
                raise KeyError(f"L'oggetto 'mrk' non esiste nel file {filename}.")
    except Exception as e:
        raise OSError(f"Errore durante l'apertura del file {filename}: {e}")
    
    print("DEBUG: Caricamento dati con h5py completato.")
    print("Dati EEG caricati correttamente.")
    print(eeg_data.shape)  # Mostra la forma dei dati EEG caricati
    
    # Continua con il resto del codice per elaborare i dati caricati
    # Ad esempio, puoi convertire eeg_data in un oggetto RawArray di MNE
    info = mne.create_info(ch_names=[f'ch{i}' for i in range(1, eeg_data.shape[0] + 1)], sfreq=250, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data, info)
    
    # Aggiungi gli eventi al raw
    events = np.array(events)
    event_id = {'Right Hand': 1, 'Left Hand': 2, 'Rest': 3, 'Feet': 4}
    # Create a new channel for the events
    stim_data = np.zeros((1, raw.n_times))
    stim_info = mne.create_info(['STI 014'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, stim_info)
    
    # Add the stim channel to the raw object
    raw.add_channels([stim_raw], force_update_info=True)
    
    # Add the events to the stim channel
    raw.add_events(events, stim_channel='STI 014')
    
    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later
    log.info("Cutting trials...")

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2]),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(raw, marker_def, clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    raw.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    log.info("Resampling...")
    raw.resample(250.0)
    log.info("Highpassing...")
    raw.filter(l_freq=low_cut_hz, h_freq=None)
    log.info("Standardizing...")
    raw.apply_function(lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                                 init_block_size=1000,
                                                                 eps=1e-4).T)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(raw, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset.X, dataset.y

# Esegui la funzione load_HGD_data
data_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset"
subject = 1
training = True
X, y = load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=True)
print(X.shape)
print(y.shape)

DEBUG: Inizio load_HGD_data
DEBUG: File da caricare: /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
DEBUG: Sensor names da caricare: ['C3', 'C4', 'C2']
DEBUG: Il file esiste.
DEBUG: Contenuto del file .mat:
ch1
ch10
ch100
ch101
ch102
ch103
ch104
ch105
ch106
ch107
ch108
ch109
ch11
ch110
ch111
ch112
ch113
ch114
ch115
ch116
ch117
ch118
ch119
ch12
ch120
ch121
ch122
ch123
ch124
ch125
ch126
ch127
ch128
ch129
ch13
ch130
ch131
ch132
ch133
ch14
ch15
ch16
ch17
ch18
ch19
ch2
ch20
ch21
ch22
ch23
ch24
ch25
ch26
ch27
ch28
ch29
ch3
ch30
ch31
ch32
ch33
ch34
ch35
ch36
ch37
ch38
ch39
ch4
ch40
ch41
ch42
ch43
ch44
ch45
ch46
ch47
ch48
ch49
ch5
ch50
ch51
ch52
ch53
ch54
ch55
ch56
ch57
ch58
ch59
ch6
ch60
ch61
ch62
ch63
ch64
ch65
ch66
ch67
ch68
ch69
ch7
ch70
ch71
ch72
ch73
ch74
ch75
ch76
ch77
ch78
ch79
ch8
ch80
ch81
ch82
ch83
ch84
ch85
ch86
ch87
ch88
ch89
ch9
ch90
ch91
ch92
ch93
ch94
ch95
ch96
ch97
ch98
ch99
dat
mnt
mrk
nfo
~obj_pointed_by_100280
~obj_pointed_b

ValueError: invalid literal for int() with base 10: '10002.0'

In [ ]:
import h5py
import numpy as np

# Carica il file .mat
filename = '/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat'
with h5py.File(filename, 'r') as f:
    # Stampa tutte le chiavi
    print("Chiavi nel file .mat:", list(f.keys()))

    # Accedi ai dati necessari
    # Assicurati che i dati siano nella forma che ti aspetti
    # Ad esempio, se i dati degli eventi sono in 'event_labels', esegui:
    event_labels = f['mrk']['event_labels'][:]  # Sostituisci con il nome giusto della variabile
    print("Event labels:", event_labels)

    # Verifica la presenza di valori non numerici
    for label in event_labels:
        if isinstance(label, str):
            print(f"Valore non numerico trovato: {label}")
    
    # A questo punto, puoi decidere come gestire i valori non numerici
    # Ad esempio, puoi rimuoverli o sostituirli
    event_labels = [label if isinstance(label, int) else 0 for label in event_labels]
    event_labels = np.array(event_labels, dtype=int)
    print("Event labels dopo la pulizia:", event_labels)


Chiavi nel file .mat: ['ch1', 'ch10', 'ch100', 'ch101', 'ch102', 'ch103', 'ch104', 'ch105', 'ch106', 'ch107', 'ch108', 'ch109', 'ch11', 'ch110', 'ch111', 'ch112', 'ch113', 'ch114', 'ch115', 'ch116', 'ch117', 'ch118', 'ch119', 'ch12', 'ch120', 'ch121', 'ch122', 'ch123', 'ch124', 'ch125', 'ch126', 'ch127', 'ch128', 'ch129', 'ch13', 'ch130', 'ch131', 'ch132', 'ch133', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch6', 'ch60', 'ch61', 'ch62', 'ch63', 'ch64', 'ch65', 'ch66', 'ch67', 'ch68', 'ch69', 'ch7', 'ch70', 'ch71', 'ch72', 'ch73', 'ch74', 'ch75', 'ch76', 'ch77', 'ch78', 'ch79', 'ch8', 'ch80', 'ch81', 'ch82', 'ch83', 'ch84', 'ch85', 'ch86',

KeyError: "Unable to open object (object 'event_labels' doesn't exist)"

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np
import scipy.io as sio
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.utils import shuffle
from torchvision import transforms

# Load High Gamma Dataset (HGD)
import numpy as np
import logging
from collections import OrderedDict
from braindecode.datasets.bbci import BBCIDataset
from braindecode.datautil.trial_segment import create_signal_target
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize
from braindecode.datautil.signalproc import highpass_cnt 
import braindecode
import torch
from torch.utils.data import Dataset

class HGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def load_HGD_data(data_path, subject, training, low_cut_hz=0, debug=False):
    log = logging.getLogger(__name__)
    log.setLevel('DEBUG')

    if training:
        filename = (data_path + 'train/{}.mat'.format(subject))
    else:
        filename = (data_path + 'test/{}.mat'.format(subject))

    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']

    loader = BBCIDataset(filename, load_sensor_names=load_sensor_names)

    log.info("Loading data...")
    cnt = loader.load()

    log.info("Cutting trials...")
    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target(cnt, marker_def, clean_ival)
    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    cnt = cnt.pick_channels(C_sensors)

    log.info("Resampling...")
    cnt = resample_cnt(cnt, 250.0)
    log.info("Highpassing...")
    cnt = mne_apply(
        lambda a: highpass_cnt(
            a, low_cut_hz, cnt.info['sfreq'], filt_order=3, axis=1),
        cnt)
    log.info("Standardizing...")
    cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T,
        cnt)

    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]

    return HGDataset(dataset.X, dataset.y)


# Define the dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = {'eeg': self.X[idx], 'label': self.y[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Custom transform to standardize the EEG data
class StandardizeEEG(object):
    def __init__(self, channels):
        self.channels = channels
        self.scalers = [StandardScaler() for _ in range(channels)]

    def __call__(self, sample):
        eeg, label = sample['eeg'], sample['label']

        for j in range(self.channels):
            eeg[:, j, :] = self.scalers[j].transform(eeg[:, j, :])

        return {'eeg': eeg, 'label': label}

# Get EEG data for PyTorch
def get_data(path, subject, dataset='HGD', classes_labels='all', LOSO=False, isStandard=True, isShuffle=True):
    # Load and split the dataset into training and testing
    if LOSO:
        # Load data using the original function
        X_train, y_train, X_test, y_test = load_data_LOSO(path, subject, dataset)
    else:
        if dataset == 'BCI2a':
            path = path + 's{:}/'.format(subject + 1)
            X_train, y_train = load_BCI2a_data(path, subject + 1, True)
            X_test, y_test = load_BCI2a_data(path, subject + 1, False)
        elif dataset == 'CS2R':
            X_train, y_train, _, _, _ = load_CS2R_data_v2(path, subject, True, classes_labels)
            X_test, y_test, _, _, _ = load_CS2R_data_v2(path, subject, False, classes_labels)
        elif dataset == 'HGD':
            # Load HGD data using the provided function
            X_train, y_train = load_HGD_data(path, subject + 1, True)
            X_test, y_test = load_HGD_data(path, subject + 1, False)
        else:
            raise Exception("'{}' dataset is not supported yet!".format(dataset))

    # Shuffle the data
    if isShuffle:
        X_train, y_train = shuffle(X_train, y_train, random_state=42)
        X_test, y_test = shuffle(X_test, y_test, random_state=42)

    # Prepare training data
    N_tr, N_ch, T = X_train.shape
    X_train = X_train.reshape(N_tr, 1, N_ch, T)
    y_train_onehot = torch.from_numpy(y_train).long()

    # Prepare testing data
    N_tr, N_ch, T = X_test.shape
    X_test = X_test.reshape(N_tr, 1, N_ch, T)
    y_test_onehot = torch.from_numpy(y_test).long()

    # Standardize the data
    if isStandard:
        channels = N_ch
        transform = transforms.Compose([StandardizeEEG(channels)])
        dataset_train = EEGDataset(X_train, y_train_onehot, transform=transform)
        dataset_test = EEGDataset(X_test, y_test_onehot, transform=transform)
    else:
        dataset_train = TensorDataset(torch.Tensor(X_train), y_train_onehot)
        dataset_test = TensorDataset(torch.Tensor(X_test), y_test_onehot)

    return dataset_train, dataset_test

# Example usage
path = '/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/'
subject = 1
dataset_train, dataset_test = get_data(path, subject, dataset='HGD', classes_labels='all', LOSO=False, isStandard=True, isShuffle=True)
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

2025-03-11 19:34:14,719 INFO : Loading data...
Caricamento dei dati da /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/2.mat
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.


AssertionError: Recheck this code if you have different sensors...

In [1]:
import logging
import sys
import os.path
from collections import OrderedDict
import numpy as np

from braindecode.datasets.bbci import  BBCIDataset
from braindecode.datautil.signalproc import highpass_cnt
import torch.nn.functional as F
import torch as th
from torch import optim
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.deep4 import Deep4Net
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.util import to_dense_prediction_model
from braindecode.experiments.experiment import Experiment
from braindecode.torch_ext.util import np_to_var
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, \
    RuntimeMonitor, CroppedTrialMisclassMonitor

from braindecode.datautil.splitters import split_into_two_sets
from braindecode.datautil.trial_segment import \
    create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize


log = logging.getLogger(__name__)
log.setLevel('DEBUG')


def load_bbci_data(filename, low_cut_hz, debug=False):
    load_sensor_names = None
    if debug:
        load_sensor_names = ['C3', 'C4', 'C2']
    # we loaded all sensors to always get same cleaning results independent of sensor selection
    # There is an inbuilt heuristic that tries to use only EEG channels and that definitely
    # works for datasets in our paper
    loader = BBCIDataset(filename, load_sensor_names=load_sensor_names)

    log.info("Loading data...")
    cnt = loader.load()

    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later
    log.info("Cutting trials...")

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(cnt, marker_def,
                                                  clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    log.info("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(clean_trial_mask),
        len(set_for_cleaning.X),
        np.mean(clean_trial_mask) * 100))

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
                 'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
                 'C6',
                 'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
                 'FCC5h',
                 'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
                 'CPP5h',
                 'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
                 'CCP1h',
                 'CCP2h', 'CPP1h', 'CPP2h']
    if debug:
        C_sensors = load_sensor_names
    cnt = cnt.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    log.info("Resampling...")
    cnt = resample_cnt(cnt, 250.0)
    log.info("Highpassing...")
    cnt = mne_apply(
        lambda a: highpass_cnt(
            a, low_cut_hz, cnt.info['sfreq'], filt_order=3, axis=1),
        cnt)
    log.info("Standardizing...")
    cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T,
        cnt)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [-500, 4000]

    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset


def load_train_valid_test(
        train_filename, test_filename, low_cut_hz, debug=False):
    log.info("Loading train...")
    full_train_set = load_bbci_data(
        train_filename, low_cut_hz=low_cut_hz, debug=debug)

    log.info("Loading test...")
    test_set = load_bbci_data(
        test_filename, low_cut_hz=low_cut_hz, debug=debug)
    valid_set_fraction = 0.8
    train_set, valid_set = split_into_two_sets(full_train_set,
                                               valid_set_fraction)

    log.info("Train set with {:4d} trials".format(len(train_set.X)))
    if valid_set is not None:
        log.info("Valid set with {:4d} trials".format(len(valid_set.X)))
    log.info("Test set with  {:4d} trials".format(len(test_set.X)))

    return train_set, valid_set, test_set


def run_exp_on_high_gamma_dataset(train_filename, test_filename,
                  low_cut_hz, model_name,
                  max_epochs, max_increase_epochs,
                  np_th_seed,
                  debug):
    input_time_length = 1000
    batch_size = 60
    lr = 1e-3
    weight_decay = 0
    train_set, valid_set, test_set = load_train_valid_test(
        train_filename=train_filename,
        test_filename=test_filename,
        low_cut_hz=low_cut_hz, debug=debug)
    if debug:
        max_epochs = 4

    set_random_seeds(np_th_seed, cuda=True)
    #torch.backends.cudnn.benchmark = True# sometimes crashes?
    n_classes = int(np.max(train_set.y) + 1)
    n_chans = int(train_set.X.shape[1])
    if model_name == 'deep':
        model = Deep4Net(n_chans, n_classes,
                         input_time_length=input_time_length,
                         final_conv_length=2).create_network()
    elif model_name == 'shallow':
        model = ShallowFBCSPNet(
            n_chans, n_classes, input_time_length=input_time_length,
            final_conv_length=30).create_network()

    to_dense_prediction_model(model)
    model.cuda()
    model.eval()

    out = model(np_to_var(train_set.X[:1, :, :input_time_length, None]).cuda())

    n_preds_per_input = out.cpu().data.numpy().shape[2]
    optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay,
                           lr=lr)

    iterator = CropsFromTrialsIterator(batch_size=batch_size,
                                       input_time_length=input_time_length,
                                       n_preds_per_input=n_preds_per_input,
                                       seed=np_th_seed)

    monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
                CroppedTrialMisclassMonitor(
                    input_time_length=input_time_length), RuntimeMonitor()]

    model_constraint = MaxNormDefaultConstraint()

    loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2),
                                                      targets)

    run_after_early_stop = True
    do_early_stop = True
    remember_best_column = 'valid_misclass'
    stop_criterion = Or([MaxEpochs(max_epochs),
                         NoDecrease('valid_misclass', max_increase_epochs)])

    exp = Experiment(model, train_set, valid_set, test_set, iterator=iterator,
                     loss_function=loss_function, optimizer=optimizer,
                     model_constraint=model_constraint,
                     monitors=monitors,
                     stop_criterion=stop_criterion,
                     remember_best_column=remember_best_column,
                     run_after_early_stop=run_after_early_stop, cuda=True,
                     do_early_stop=do_early_stop)
    exp.run()
    return exp


if __name__ == '__main__':
    logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
    subject_id = 1
    # have to change the data_folder here to make it run.
    data_folder = '/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset'
    train_filename =  os.path.join(
        data_folder, 'train/{:d}.mat'.format(subject_id))
    test_filename =  os.path.join(
        data_folder, 'test/{:d}.mat'.format(subject_id))
    max_epochs = 800
    max_increase_epochs = 80
    model_name = 'deep' # or shallow
    low_cut_hz = 0 # or 4
    np_th_seed = 0 # random seed for numpy and pytorch
    debug = False
    exp = run_exp_on_high_gamma_dataset(train_filename, test_filename,
                                  low_cut_hz, model_name,
                                  max_epochs, max_increase_epochs,
                                  np_th_seed,
                                  debug)
    log.info("Last 10 epochs")
    log.info("\n" + str(exp.epochs_df.iloc[-10:]))

2025-03-11 18:29:08,003 INFO : Loading train...
2025-03-11 18:29:08,003 INFO : Loading data...
Caricamento dei dati da /Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non riconosciuto per <HDF5 object reference>, lo salto.
Formato non rico

AssertionError: Recheck this code if you have different sensors...

In [3]:
import h5py

file_path = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(file_path, 'r') as f:
    def print_hdf5_structure(name, obj):
        print(name)
    
    f.visititems(print_hdf5_structure)  # Stampa la struttura del file


ch1
ch10
ch100
ch101
ch102
ch103
ch104
ch105
ch106
ch107
ch108
ch109
ch11
ch110
ch111
ch112
ch113
ch114
ch115
ch116
ch117
ch118
ch119
ch12
ch120
ch121
ch122
ch123
ch124
ch125
ch126
ch127
ch128
ch129
ch13
ch130
ch131
ch132
ch133
ch14
ch15
ch16
ch17
ch18
ch19
ch2
ch20
ch21
ch22
ch23
ch24
ch25
ch26
ch27
ch28
ch29
ch3
ch30
ch31
ch32
ch33
ch34
ch35
ch36
ch37
ch38
ch39
ch4
ch40
ch41
ch42
ch43
ch44
ch45
ch46
ch47
ch48
ch49
ch5
ch50
ch51
ch52
ch53
ch54
ch55
ch56
ch57
ch58
ch59
ch6
ch60
ch61
ch62
ch63
ch64
ch65
ch66
ch67
ch68
ch69
ch7
ch70
ch71
ch72
ch73
ch74
ch75
ch76
ch77
ch78
ch79
ch8
ch80
ch81
ch82
ch83
ch84
ch85
ch86
ch87
ch88
ch89
ch9
ch90
ch91
ch92
ch93
ch94
ch95
ch96
ch97
ch98
ch99
dat
dat/yUnit
mnt
mnt/box
mnt/box_sz
mnt/clab
mnt/pos_3d
mnt/scale_box
mnt/scale_box_sz
mnt/x
mnt/y
mrk
mrk/className
mrk/event
mrk/event/desc
mrk/time
mrk/y
nfo
nfo/T
nfo/clab
nfo/className
nfo/fs
~obj_pointed_by_100280
~obj_pointed_by_100592
~obj_pointed_by_100904
~obj_pointed_by_101216
~obj_pointed_by_1018

In [4]:
with h5py.File(file_path, 'r') as f:
    if 'sensor_names' in f:
        sensor_names = f['sensor_names'][()]
        print("Nomi dei sensori:", sensor_names)
    else:
        print("Il dataset 'sensor_names' non è presente nel file HDF5.")


Il dataset 'sensor_names' non è presente nel file HDF5.


In [5]:
with h5py.File(file_path, 'r') as f:
    for key in f.keys():
        print(f"Dataset: {key}, Shape: {f[key].shape}")


Dataset: ch1, Shape: (1, 1225545)
Dataset: ch10, Shape: (1, 1225545)
Dataset: ch100, Shape: (1, 1225545)
Dataset: ch101, Shape: (1, 1225545)
Dataset: ch102, Shape: (1, 1225545)
Dataset: ch103, Shape: (1, 1225545)
Dataset: ch104, Shape: (1, 1225545)
Dataset: ch105, Shape: (1, 1225545)
Dataset: ch106, Shape: (1, 1225545)
Dataset: ch107, Shape: (1, 1225545)
Dataset: ch108, Shape: (1, 1225545)
Dataset: ch109, Shape: (1, 1225545)
Dataset: ch11, Shape: (1, 1225545)
Dataset: ch110, Shape: (1, 1225545)
Dataset: ch111, Shape: (1, 1225545)
Dataset: ch112, Shape: (1, 1225545)
Dataset: ch113, Shape: (1, 1225545)
Dataset: ch114, Shape: (1, 1225545)
Dataset: ch115, Shape: (1, 1225545)
Dataset: ch116, Shape: (1, 1225545)
Dataset: ch117, Shape: (1, 1225545)
Dataset: ch118, Shape: (1, 1225545)
Dataset: ch119, Shape: (1, 1225545)
Dataset: ch12, Shape: (1, 1225545)
Dataset: ch120, Shape: (1, 1225545)
Dataset: ch121, Shape: (1, 1225545)
Dataset: ch122, Shape: (1, 1225545)
Dataset: ch123, Shape: (1, 122554

AttributeError: 'Group' object has no attribute 'shape'

In [6]:
filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"
with h5py.File(filename, 'r') as h5file:
    if 'mnt' in h5file and 'clab' in h5file['mnt']:
        clab_set = h5file['mnt']['clab'][:]
        print("Tipo dei dati in 'clab':", type(clab_set))
        print("Esempio di dati:", clab_set[:10])  # Mostra i primi 10 valori
    else:
        print("Errore: Il file non contiene 'mnt/clab'")


Tipo dei dati in 'clab': <class 'numpy.ndarray'>
Esempio di dati: [[<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]
 [<HDF5 object reference>]]


In [13]:
import h5py
import numpy as np

filename = "/Users/giuseppebonomo/Desktop/UNIPA CdLM Ing_Informatica/Tesi/cod/EEG-ATCNet/dataset/train/1.mat"

with h5py.File(filename, 'r') as h5file:
    try:
        clab_set = h5file['mnt']['clab'][:].squeeze()
    except KeyError:
        print(f"Errore: il file {filename} non contiene la chiave 'mnt/clab'.")
        all_sensor_names = []
    else:
        all_sensor_names = []

        for obj in clab_set:
            if isinstance(obj, bytes):  
                all_sensor_names.append(obj.decode("utf-8"))
            elif isinstance(obj, np.bytes_):  
                all_sensor_names.append(obj.astype(str))
            elif isinstance(obj, np.ndarray):  
                all_sensor_names.extend(obj.astype(str))
            elif isinstance(obj, (int, np.integer)):  
                all_sensor_names.append(chr(obj))
            elif isinstance(obj, str):  
                all_sensor_names.append(obj)
            elif isinstance(obj, h5py.h5r.Reference):  # Se è un riferimento HDF5
                ref_obj = h5file[obj]  # Segui il riferimento
                if isinstance(ref_obj, h5py.Dataset):  # Se il riferimento punta a un dataset
                    all_sensor_names.append(ref_obj[()])  # Leggi il valore
                else:
                    print(f"Riferimento non riconosciuto per {obj}, tipo {type(ref_obj)}")
            else:
                print(f"Formato non riconosciuto per {obj}, tipo {type(obj)}")

    print(all_sensor_names)
    
# Decodifica gli array
decoded_strings = ["".join(chr(c[0]) for c in arr) for arr in all_sensor_names]

# Stampa le stringhe
for s in decoded_strings:
    print(s)


[array([[ 70],
       [112],
       [ 49]], dtype=uint16), array([[ 70],
       [112],
       [ 50]], dtype=uint16), array([[ 70],
       [112],
       [122]], dtype=uint16), array([[70],
       [55]], dtype=uint16), array([[70],
       [51]], dtype=uint16), array([[ 70],
       [122]], dtype=uint16), array([[70],
       [52]], dtype=uint16), array([[70],
       [56]], dtype=uint16), array([[70],
       [67],
       [53]], dtype=uint16), array([[70],
       [67],
       [49]], dtype=uint16), array([[70],
       [67],
       [50]], dtype=uint16), array([[70],
       [67],
       [54]], dtype=uint16), array([[77],
       [49]], dtype=uint16), array([[84],
       [55]], dtype=uint16), array([[67],
       [51]], dtype=uint16), array([[ 67],
       [122]], dtype=uint16), array([[67],
       [52]], dtype=uint16), array([[84],
       [56]], dtype=uint16), array([[77],
       [50]], dtype=uint16), array([[67],
       [80],
       [53]], dtype=uint16), array([[67],
       [80],
       [49]], dt